In [2]:
import pandas as pd
import numpy as np
import os, zipfile
import subprocess

In [95]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import lightgbm as lgbm
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [24]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [25]:
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

In [15]:
from imblearn.under_sampling import RandomUnderSampler

In [3]:
import category_encoders as ce

In [6]:
dir_name = 'D:\\1mfti\ml\Home-Credit-Default-Risk'
os.chdir(dir_name)
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [9]:
num_cols = list(train.select_dtypes(include=['int64', 'float64']).columns)
num_cols.remove('TARGET')

In [12]:
cat_cols = list(train.select_dtypes(include=['object']).columns)

In [16]:
def preprocess(train1, test1):
    train1 = train1.fillna(-10)
    test1 = test1.fillna(-10)
    train1[num_cols] = train1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    test1[num_cols] = test1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    WOE_encoder = ce.WOEEncoder()
    train1[cat_cols] = WOE_encoder.fit_transform(train1[cat_cols], train1['TARGET'])
    test1[cat_cols] = WOE_encoder.transform(train1[cat_cols])

    return train1, test1

In [73]:
train_norm, test_norm = preprocess(train, test)

C:\Python\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [74]:
X_train = train_norm.drop('TARGET', axis=1)
y_train = train_norm['TARGET']
X_test = test_norm

In [75]:
ada = RandomUnderSampler(random_state=42)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [29]:
# X_res, y_res = X_res[:20000], y_res[:20000]

In [76]:
X_res, X_valid, y_res, y_valid = train_test_split(X_res, y_res,
                                                    stratify=y_res, 
                                                    test_size=0.2)

In [65]:
lgbm_clf = lgbm.LGBMClassifier(objective='binary', 
         boosting_type = 'goss', 
         n_estimators = 50, class_weight ='balanced', learning_rate =0.01, random_state=42, eval_set=[X_valid, y_valid])

In [66]:
lgbm_clf.fit(X_res, y_res)

LGBMClassifier(boosting_type='goss', class_weight='balanced',
               eval_set=[        SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  \
143886           1            0.036202    -0.154315      0.056221   
12347            0            0.036202     0.250941     -0.117305   
6867             0            0.036202    -0.154315     -0.117305   
93346            1            0.036202     0.250941      0.056221   
182281           2            0.036202    -0.154315      0.056221   
...            ...                 ...          ...           ...   
167301           2            0.036202    -0.154315     -0.117305   
50257            0            0.036202    -0.154315     -...
182281                          1                          0   
...                           ...                        ...   
167301                          1                          0   
50257                           1                          0   
171000                          1           

In [ ]:
# Точность на kaggle:0.57896 learning_rate=0.01, n_estimators=50, objective='binary',
               random_state=42)

In [88]:


clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    n
    
    #loss_function='CrossEntropy'
)
#Method 1
grid = dict()
grid['n_estimators'] = [100, 200, 300]
grid['max_depth'] = [3,4,5]
Method 2
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}
#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='accuracy', cv = 5)


clf.fit(X_res, y_res, 
        
        eval_set=(X_valid, y_valid), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [109]:

cbc = CatBoostClassifier()

# Creating the hyperparameter grid
param_dist = { "learning_rate": np.linspace(0,0.2,5),
               "max_depth": sp_randint(3, 10),
            "learning_rate": [0.1, 0.01, 0.2, 0.3]},
    
               
#Instantiate RandomSearchCV object
rscv = RandomizedSearchCV(cbc , param_dist, scoring='roc_auc', cv =5)

#Fit the model
rscv.fit(X_res,y_res)

# Print the tuned parameters and score
print(rscv.best_params_)
print(rscv.best_score_)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.6776889	total: 18.9ms	remaining: 18.9s
1:	learn: 0.6657481	total: 45.8ms	remaining: 22.9s
2:	learn: 0.6574149	total: 76.4ms	remaining: 25.4s
3:	learn: 0.6524775	total: 96.3ms	remaining: 24s
4:	learn: 0.6488542	total: 116ms	remaining: 23.1s
5:	learn: 0.6456677	total: 135ms	remaining: 22.3s
6:	learn: 0.6433524	total: 162ms	remaining: 23s
7:	learn: 0.6413230	total: 180ms	remaining: 22.3s
8:	learn: 0.6398229	total: 198ms	remaining: 21.7s
9:	learn: 0.6388771	total: 215ms	remaining: 21.3s
10:	learn: 0.6370983	total: 235ms	remaining: 21.2s
11:	learn: 0.6360140	total: 254ms	remaining: 20.9s
12:	learn: 0.6350144	total: 272ms	remaining: 20.6s
13:	learn: 0.6339237	total: 289ms	remaining: 20.4s
14:	learn: 0.6332125	total: 306ms	remaining: 20.1s
15:	learn: 0.6321359	total: 323ms	remaining: 19.9s
16:	learn: 0.6309178	total: 341ms	remaining: 19.7s
17:	learn: 0.6300429	total: 367ms	remaining: 20s
18:	learn: 0.6288042	total: 397ms	remaining: 20.5s
19:	learn: 0.6282624	total: 414ms	remaining

169:	learn: 0.5316415	total: 3.48s	remaining: 17s
170:	learn: 0.5311254	total: 3.51s	remaining: 17s
171:	learn: 0.5308607	total: 3.53s	remaining: 17s
172:	learn: 0.5301800	total: 3.55s	remaining: 17s
173:	learn: 0.5298490	total: 3.59s	remaining: 17.1s
174:	learn: 0.5295965	total: 3.62s	remaining: 17.1s
175:	learn: 0.5289274	total: 3.64s	remaining: 17.1s
176:	learn: 0.5285591	total: 3.66s	remaining: 17s
177:	learn: 0.5280392	total: 3.68s	remaining: 17s
178:	learn: 0.5278037	total: 3.7s	remaining: 17s
179:	learn: 0.5272855	total: 3.72s	remaining: 16.9s
180:	learn: 0.5271780	total: 3.73s	remaining: 16.9s
181:	learn: 0.5269308	total: 3.75s	remaining: 16.9s
182:	learn: 0.5267881	total: 3.77s	remaining: 16.8s
183:	learn: 0.5264479	total: 3.79s	remaining: 16.8s
184:	learn: 0.5259404	total: 3.81s	remaining: 16.8s
185:	learn: 0.5253243	total: 3.83s	remaining: 16.7s
186:	learn: 0.5249481	total: 3.84s	remaining: 16.7s
187:	learn: 0.5244266	total: 3.86s	remaining: 16.7s
188:	learn: 0.5235742	total

335:	learn: 0.4580198	total: 6.96s	remaining: 13.8s
336:	learn: 0.4575369	total: 6.99s	remaining: 13.7s
337:	learn: 0.4571018	total: 7s	remaining: 13.7s
338:	learn: 0.4566879	total: 7.02s	remaining: 13.7s
339:	learn: 0.4559070	total: 7.04s	remaining: 13.7s
340:	learn: 0.4557170	total: 7.06s	remaining: 13.6s
341:	learn: 0.4554452	total: 7.08s	remaining: 13.6s
342:	learn: 0.4551517	total: 7.09s	remaining: 13.6s
343:	learn: 0.4547425	total: 7.11s	remaining: 13.6s
344:	learn: 0.4544723	total: 7.13s	remaining: 13.5s
345:	learn: 0.4542143	total: 7.15s	remaining: 13.5s
346:	learn: 0.4540484	total: 7.17s	remaining: 13.5s
347:	learn: 0.4537170	total: 7.18s	remaining: 13.5s
348:	learn: 0.4534106	total: 7.2s	remaining: 13.4s
349:	learn: 0.4531934	total: 7.22s	remaining: 13.4s
350:	learn: 0.4530116	total: 7.24s	remaining: 13.4s
351:	learn: 0.4525699	total: 7.27s	remaining: 13.4s
352:	learn: 0.4525448	total: 7.28s	remaining: 13.4s
353:	learn: 0.4523316	total: 7.3s	remaining: 13.3s
354:	learn: 0.451

500:	learn: 0.3997897	total: 10.5s	remaining: 10.4s
501:	learn: 0.3994082	total: 10.5s	remaining: 10.4s
502:	learn: 0.3991240	total: 10.5s	remaining: 10.4s
503:	learn: 0.3988953	total: 10.5s	remaining: 10.4s
504:	learn: 0.3983790	total: 10.6s	remaining: 10.4s
505:	learn: 0.3981359	total: 10.6s	remaining: 10.3s
506:	learn: 0.3977313	total: 10.6s	remaining: 10.3s
507:	learn: 0.3972294	total: 10.6s	remaining: 10.3s
508:	learn: 0.3968376	total: 10.6s	remaining: 10.3s
509:	learn: 0.3964971	total: 10.7s	remaining: 10.2s
510:	learn: 0.3961809	total: 10.7s	remaining: 10.2s
511:	learn: 0.3960297	total: 10.7s	remaining: 10.2s
512:	learn: 0.3957299	total: 10.7s	remaining: 10.2s
513:	learn: 0.3955070	total: 10.7s	remaining: 10.2s
514:	learn: 0.3951637	total: 10.8s	remaining: 10.1s
515:	learn: 0.3946645	total: 10.8s	remaining: 10.1s
516:	learn: 0.3944616	total: 10.8s	remaining: 10.1s
517:	learn: 0.3939171	total: 10.8s	remaining: 10.1s
518:	learn: 0.3934347	total: 10.8s	remaining: 10.1s
519:	learn: 

...

268:	learn: 0.6352899	total: 7.38s	remaining: 20s
269:	learn: 0.6352124	total: 7.4s	remaining: 20s
270:	learn: 0.6351512	total: 7.43s	remaining: 20s
271:	learn: 0.6350849	total: 7.45s	remaining: 19.9s
272:	learn: 0.6350186	total: 7.47s	remaining: 19.9s
273:	learn: 0.6349461	total: 7.5s	remaining: 19.9s
274:	learn: 0.6348570	total: 7.53s	remaining: 19.9s
275:	learn: 0.6347939	total: 7.55s	remaining: 19.8s
276:	learn: 0.6347027	total: 7.58s	remaining: 19.8s
277:	learn: 0.6346139	total: 7.61s	remaining: 19.8s
278:	learn: 0.6345298	total: 7.63s	remaining: 19.7s
279:	learn: 0.6344619	total: 7.66s	remaining: 19.7s
280:	learn: 0.6344013	total: 7.68s	remaining: 19.7s
281:	learn: 0.6343077	total: 7.71s	remaining: 19.6s
282:	learn: 0.6342344	total: 7.74s	remaining: 19.6s
283:	learn: 0.6341691	total: 7.76s	remaining: 19.6s
284:	learn: 0.6340748	total: 7.8s	remaining: 19.6s
285:	learn: 0.6339842	total: 7.83s	remaining: 19.5s
286:	learn: 0.6339365	total: 7.85s	remaining: 19.5s
287:	learn: 0.6338681

434:	learn: 0.6257580	total: 12s	remaining: 15.5s
435:	learn: 0.6257144	total: 12s	remaining: 15.5s
436:	learn: 0.6256670	total: 12s	remaining: 15.5s
437:	learn: 0.6256321	total: 12s	remaining: 15.5s
438:	learn: 0.6255770	total: 12.1s	remaining: 15.4s
439:	learn: 0.6255419	total: 12.1s	remaining: 15.4s
440:	learn: 0.6255089	total: 12.1s	remaining: 15.4s
441:	learn: 0.6254298	total: 12.2s	remaining: 15.3s
442:	learn: 0.6253530	total: 12.2s	remaining: 15.3s
443:	learn: 0.6253013	total: 12.2s	remaining: 15.3s
444:	learn: 0.6252735	total: 12.2s	remaining: 15.3s
445:	learn: 0.6252235	total: 12.3s	remaining: 15.2s
446:	learn: 0.6251702	total: 12.3s	remaining: 15.2s
447:	learn: 0.6251222	total: 12.3s	remaining: 15.2s
448:	learn: 0.6250787	total: 12.3s	remaining: 15.2s
449:	learn: 0.6250147	total: 12.4s	remaining: 15.1s
450:	learn: 0.6249749	total: 12.4s	remaining: 15.1s
451:	learn: 0.6249384	total: 12.4s	remaining: 15.1s
452:	learn: 0.6248972	total: 12.5s	remaining: 15s
453:	learn: 0.6248695	

600:	learn: 0.6187010	total: 16.4s	remaining: 10.9s
601:	learn: 0.6186768	total: 16.4s	remaining: 10.8s
602:	learn: 0.6186303	total: 16.4s	remaining: 10.8s
603:	learn: 0.6186052	total: 16.4s	remaining: 10.8s
604:	learn: 0.6185663	total: 16.5s	remaining: 10.7s
605:	learn: 0.6185413	total: 16.5s	remaining: 10.7s
606:	learn: 0.6185056	total: 16.5s	remaining: 10.7s
607:	learn: 0.6184779	total: 16.5s	remaining: 10.7s
608:	learn: 0.6184398	total: 16.6s	remaining: 10.6s
609:	learn: 0.6184082	total: 16.6s	remaining: 10.6s
610:	learn: 0.6183631	total: 16.6s	remaining: 10.6s
611:	learn: 0.6183427	total: 16.6s	remaining: 10.5s
612:	learn: 0.6182926	total: 16.7s	remaining: 10.5s
613:	learn: 0.6182759	total: 16.7s	remaining: 10.5s
614:	learn: 0.6182460	total: 16.7s	remaining: 10.5s
615:	learn: 0.6182197	total: 16.7s	remaining: 10.4s
616:	learn: 0.6181791	total: 16.8s	remaining: 10.4s
617:	learn: 0.6181309	total: 16.8s	remaining: 10.4s
618:	learn: 0.6180888	total: 16.8s	remaining: 10.3s
619:	learn: 

765:	learn: 0.6129393	total: 20.7s	remaining: 6.33s
766:	learn: 0.6128948	total: 20.8s	remaining: 6.3s
767:	learn: 0.6128555	total: 20.8s	remaining: 6.28s
768:	learn: 0.6128242	total: 20.8s	remaining: 6.25s
769:	learn: 0.6127935	total: 20.8s	remaining: 6.22s
770:	learn: 0.6127734	total: 20.9s	remaining: 6.19s
771:	learn: 0.6127443	total: 20.9s	remaining: 6.17s
772:	learn: 0.6127144	total: 20.9s	remaining: 6.14s
773:	learn: 0.6126780	total: 20.9s	remaining: 6.11s
774:	learn: 0.6126291	total: 21s	remaining: 6.08s
775:	learn: 0.6126047	total: 21s	remaining: 6.06s
776:	learn: 0.6125858	total: 21s	remaining: 6.03s
777:	learn: 0.6125676	total: 21s	remaining: 6s
778:	learn: 0.6125421	total: 21.1s	remaining: 5.98s
779:	learn: 0.6125247	total: 21.1s	remaining: 5.95s
780:	learn: 0.6124932	total: 21.1s	remaining: 5.92s
781:	learn: 0.6124730	total: 21.1s	remaining: 5.89s
782:	learn: 0.6124230	total: 21.2s	remaining: 5.86s
783:	learn: 0.6123856	total: 21.2s	remaining: 5.84s
784:	learn: 0.6123410	to

930:	learn: 0.6075897	total: 25.1s	remaining: 1.86s
931:	learn: 0.6075792	total: 25.1s	remaining: 1.83s
932:	learn: 0.6075723	total: 25.1s	remaining: 1.8s
933:	learn: 0.6075399	total: 25.1s	remaining: 1.78s
934:	learn: 0.6074871	total: 25.2s	remaining: 1.75s
935:	learn: 0.6074719	total: 25.2s	remaining: 1.72s
936:	learn: 0.6074385	total: 25.2s	remaining: 1.7s
937:	learn: 0.6074165	total: 25.2s	remaining: 1.67s
938:	learn: 0.6073802	total: 25.3s	remaining: 1.64s
939:	learn: 0.6073553	total: 25.3s	remaining: 1.61s
940:	learn: 0.6073285	total: 25.3s	remaining: 1.59s
941:	learn: 0.6072728	total: 25.4s	remaining: 1.56s
942:	learn: 0.6072320	total: 25.4s	remaining: 1.53s
943:	learn: 0.6072085	total: 25.4s	remaining: 1.51s
944:	learn: 0.6071556	total: 25.4s	remaining: 1.48s
945:	learn: 0.6071157	total: 25.5s	remaining: 1.45s
946:	learn: 0.6070892	total: 25.5s	remaining: 1.43s
947:	learn: 0.6070433	total: 25.5s	remaining: 1.4s
948:	learn: 0.6069956	total: 25.5s	remaining: 1.37s
949:	learn: 0.6

In [110]:
print(rscv.best_params_)
print(rscv.best_score_)

{'learning_rate': 0.01, 'max_depth': 8}
0.6725260167091087


In [ ]:
#Точность на kaggle: 0.57147  

In [79]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('bbc', BalancedBaggingClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    # level0.append(('lama', automl))
    level0.append(('lgbm', lgbm_clf))
    # define meta learner model
    level1 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [54]:
def get_kaggle_results():
    p = subprocess.Popen("kaggle competitions submissions  -c home-credit-default-risk", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print(output.decode())

In [80]:
stacking = get_stacking()

In [81]:
stacking.fit(X_res, y_res)

StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(C=0.0001,
                                                   class_weight='balanced',
                                                   l1_ratio=0.01, max_iter=200,
                                                   penalty='elasticnet',
                                                   solver='saga')),
                               ('knn', KNeighborsClassifier()),
                               ('bbc', BalancedBaggingClassifier()),
                               ('svm', SVC()), ('bayes', GaussianNB()),
                               ('lgbm',
                                LGBMClassifier(boosting_type='goss',
                                               class_weight='balanced',
                                               eval_set=[        SK_ID_CURR...
171000                           1  
221813                           1  
270975                           0  

[6

In [92]:
predict_cat = clf.predict(X_test)
write_submission_2['TARGET'] = predict_cat
write_submission_2.to_csv('predict_cat.csv', index= False)

In [82]:
predict_2 = stacking.predict(X_test)

In [83]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_stacking.csv', index= False)

In [93]:
!kaggle competitions submit -c home-credit-default-risk -f predict_cat.csv -m "lgbm"

Successfully submitted to Home Credit Default Risk



  0%|          | 0.00/476k [00:00<?, ?B/s]
  2%|1         | 8.00k/476k [00:00<00:13, 36.7kB/s]
 20%|##        | 96.0k/476k [00:00<00:01, 358kB/s] 
 30%|###       | 144k/476k [00:00<00:00, 349kB/s] 
 45%|####5     | 216k/476k [00:00<00:00, 442kB/s]
 81%|########  | 384k/476k [00:00<00:00, 798kB/s]
100%|##########| 476k/476k [00:00<00:00, 597kB/s]
100%|##########| 476k/476k [00:03<00:00, 157kB/s]


In [94]:
get_kaggle_results()

fileName                          date                 description  status    publicScore  privateScore  
--------------------------------  -------------------  -----------  --------  -----------  ------------  
predict_cat.csv                   2021-12-02 20:14:03  lgbm         complete  0.56172      0.57147       
predict_stacking.csv              2021-12-02 20:09:50  lgbm         complete  0.53894      0.53890       
predict_lbgm.csv                  2021-12-02 19:23:30  lgbm         complete  0.50000      0.50000       
predict_lbgm.csv                  2021-12-02 19:21:42  lgbm         complete  0.58216      0.57896       
predict_under_sample.csv          2021-12-02 19:20:48  lgbm         complete  0.58886      0.59255       
predict_under_sample.csv          2021-12-02 18:44:24  Message      complete  0.58886      0.59255       
predict_under_sample.csv          2021-12-02 18:43:59  Message      complete  0.60274      0.58831       
predict_under_sample.csv          2021-12-02 1